<a href="https://colab.research.google.com/github/anaisidoro/ASOCIO-2025---POwer-Girls/blob/main/Instance8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import json
!pip install pulp
from pulp import *
import time
# --- INÍCIO DA MEDIÇÃO ---
start_time = time.time()

In [8]:
#Input data
with open('/content/instance8 (1).json') as f:
    instance = json.load(f)

#Sets and parameters
days = ["L", "Ma", "Mi", "J", "V"]
employee = instance['Employees']
desks = instance['Desks']
zones = instance['Zones']
desks_per_zone = instance['Desks_Z']
desks_per_employee = instance['Desks_E']
employee_per_group = instance['Employees_G']
preferable_days = instance['Days_E']
weight = 5
groups = [g for g in employee_per_group]

#Update weight matrix
weights = [[1 for _ in employee] for _ in days]
for e in employee:
    for i in range(len(preferable_days[e])):
        day_index = days.index(preferable_days[e][i])
        weights[day_index][employee.index(e)]=weight

#Identify the employees that can use each desk
employee_per_desk = {}
for desk_id in desks:
    employees_in_desk = []
    for e in employee:
        possible_desks = desks_per_employee[e]
        for m in possible_desks:
            if m == desk_id:
                employees_in_desk.append(e)
    employee_per_desk[desk_id]=employees_in_desk

In [9]:
#Problem Initialization
assignment = LpProblem("StudentChallenge", LpMaximize)

#Decision Variables
x = LpVariable.dicts("x", [(d,e,k) for d in days for e in employee for k in desks if e in employee_per_desk[k]], cat = "Binary")
u = LpVariable.dicts("u", [(d,g) for d in days for g in groups], cat = "Binary")

#Objective Function
assignment += lpSum([weights[days.index(d)][employee.index(e)]*x[d,e, k] for d in days for e in employee for k in desks if e in employee_per_desk[k]])

#Constraint 1 = Each employee must stay at the office at least twice a week
for e in employee:
    assignment += lpSum([x[d,e,k] for d in days for k in desks if e in employee_per_desk[k]]) >= 2

#Constraint 2 = Each desk must be occupied by at most one specific employees at each day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee_per_desk[k]]) <= 1

#Constraint 3 - Each employee must be assigned at most once a day
for d in days:
    for e in employee:
        assignment += lpSum([x[d,e,k] for k in desks if e in employee_per_desk[k]]) <= 1

#Constraint 4 - Each desk must be assigned at most once a day
for k in desks:
    for d in days:
        assignment += lpSum([x[d,e,k] for e in employee if e in employee_per_desk[k]]) <= 1


#Constraint 5 = Employees should not be isolated in a zone
for d in days:
    for z in zones:
        for g in groups:
            for m in employee_per_group[g]:
                assignment += lpSum([x[d,m,k] for k in desks_per_employee[m] if k in desks_per_zone[z]]) <= lpSum([x[d,e,k] for e in employee_per_group[g] if e!=m for k in desks_per_employee[e] if k in desks_per_zone[z]])


#Constraint 6 = Each group should have one meeting per week
for g in groups:
    assignment += lpSum([u[d,g] for d in days]) >= 1

#Constraint 7 = All the members of a group should be present once a week
for d in days:
    for g in groups:
        employees_group = employee_per_group[g]
        assignment += lpSum([x[d,e,k] for e in employees_group for k in desks if e in employee_per_desk[k]]) >= len(employees_group)*u[d,g]

In [10]:
#Solving the model
solve_the_model = assignment.solve(PULP_CBC_CMD(timeLimit=3600, keepFiles=True))
print(f"Status do problema: {LpStatus[solve_the_model]}")

#Print variables
for var in assignment.variables():
    if var.varValue > 0:
        print(f"{var.name}={var.varValue}")

#Print objective function value
print(f"Total Happiness: {value(assignment.objective)}")

Status do problema: Not Solved
u_('J',_'G1')=0.85714286
u_('J',_'G10')=0.5
u_('L',_'G0')=0.68498168
u_('L',_'G10')=0.48931624
u_('L',_'G11')=0.33333333
u_('L',_'G2')=0.61172161
u_('L',_'G3')=0.35668498
u_('L',_'G4')=0.57142857
u_('L',_'G5')=0.28571429
u_('L',_'G6')=0.86492674
u_('Ma',_'G8')=0.40979255
u_('Mi',_'G0')=0.31501832
u_('Mi',_'G11')=0.66666667
u_('Mi',_'G2')=0.38827839
u_('Mi',_'G3')=0.56781851
u_('Mi',_'G5')=0.37767493
u_('Mi',_'G7')=0.80380037
u_('Mi',_'G8')=0.59020745
u_('Mi',_'G9')=0.5
u_('V',_'G1')=0.14285714
u_('V',_'G10')=0.010683761
u_('V',_'G3')=0.075496509
u_('V',_'G4')=0.42857143
u_('V',_'G5')=0.33661079
u_('V',_'G6')=0.13507326
u_('V',_'G7')=0.19619963
u_('V',_'G9')=0.83333333
x_('J',_'E10',_'D13')=0.25
x_('J',_'E10',_'D20')=0.43571429
x_('J',_'E10',_'D39')=0.31428571
x_('J',_'E11',_'D27')=0.25714286
x_('J',_'E11',_'D6')=0.74285714
x_('J',_'E12',_'D32')=0.035714286
x_('J',_'E12',_'D37')=0.25714286
x_('J',_'E12',_'D7')=0.70714286
x_('J',_'E13',_'D22')=0.77142857
x_

In [11]:
#Print results organized per day
for d in days:
    print(f"-------------------Day: {d}----------------------")
    for k in desks:
        for e in employee:
            if e in employee_per_desk[k]:
                if x[d, e, k].value() == 1:
                    print(f"Desk: {k}, employee:{e}")
    for g in groups:
        if u[d, g].value() == 1:
            group_employees = employee_per_group[g]
            print(f"Meeting of the day: Group {g} - {group_employees}")

-------------------Day: L----------------------
-------------------Day: Ma----------------------
-------------------Day: Mi----------------------
-------------------Day: J----------------------
-------------------Day: V----------------------


In [12]:
# --- FIM DA MEDIÇÃO ---
end_time = time.time()

# Calcula o tempo total decorrido
time_taken = end_time - start_time

# Imprime o tempo de execução formatado
print(f"\n--- Tempo de Execução do Código ---")
print(f"O processo completo levou {time_taken:.4f} segundos para ser executado.")
print(f"-----------------------------------")


--- Tempo de Execução do Código ---
O processo completo levou 298.8780 segundos para ser executado.
-----------------------------------
